<a href="https://colab.research.google.com/github/ZLY1223/Vaccine_LLM_Prompt_Engineering/blob/main/TextGrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxx"

In [ ]:
!pip install textgrad

import os
import argparse
import concurrent
from dotenv import load_dotenv
from tqdm import tqdm
import textgrad as tg
from textgrad.tasks import load_task
import numpy as np
import random
import pandas as pd
from textgrad.engine import EngineLM
from textgrad.autograd.string_based_ops import StringBasedFunction
from textgrad.loss import MultiFieldTokenParsedEvaluation
from typing import Callable, Tuple
import re

load_dotenv(override=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.3/786.3 kB 39.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.99.1
    Uninstalling openai-1.99.1:
      Successfully uninstalled openai-1.99.1


False

In [ ]:
import os
import re
import logging
import random
import concurrent.futures
from typing import List, Tuple, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import textgrad as tg
from textgrad.tasks import DataLoader
from textgrad.autograd.string_based_ops import StringBasedFunction

# =========================
# 0. Basic settings
# =========================
logging.getLogger("textgrad").setLevel(logging.WARNING)
logging.getLogger("transformers").setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.WARNING)

RANDOM_SEED = 42
BATCH_SIZE = 3
EPOCHS = 10
MAX_STEPS_PER_EPOCH = 3
VAL_SPLIT_SAVE = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/RAG/val_ids_250.csv"
TEST_SPLIT_SAVE = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/RAG/test_ids_250.csv"
BEST_PROMPT_PATH = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/RAG/best_prompt_250.txt"
BEST_ACCURACY_PATH = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/RAG/best_accuracy_250.txt"
BEST_FULL_PROMPT_PATH = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/RAG/best_full_prompt_250.txt"

RESULT_PATH = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt3.5/M2/Media_four_diets/Task_Results_Two_Options_Four_Diets_M2_4omini_seed1000.xlsx"
TRUTH_PATH = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/test_500/Chatgpt/Three_Task_Original_Response.xlsx"

EVAL_TARGET = 'accuracy'


def set_seed(seed: int = 42):
    np.random.seed(seed)
    random.seed(seed)

set_seed(RANDOM_SEED)

# =========================
# 1. Prompt fragment security segmentation
# =========================
TASK_TAG = "% Task Prompt"
SPLIT_TOK = "---\nOutput format:"
SECONDARY_SPLIT = "You must strictly follow this output format:"

CANONICAL_PREFIX = ""
CANONICAL_SUFFIX = ""

def safe_split(raw_prompt: str) -> Tuple[Optional[str], Optional[str], Optional[Tuple[int, int]]]:
    if not isinstance(raw_prompt, str):
        return None, None, None
    tag_idx = raw_prompt.find(TASK_TAG)
    split_idx = raw_prompt.find(SPLIT_TOK)
    if tag_idx == -1 or split_idx == -1 or split_idx <= tag_idx:
        return None, None, None
    prefix = raw_prompt[:tag_idx + len(TASK_TAG)].strip() + "\n"
    suffix = raw_prompt[split_idx:].strip()
    return prefix, suffix, (tag_idx, split_idx)


def extract_middle_trainable(raw_prompt: str) -> str:
    """Extract trainable segments between `% Task Prompt` and `Output format` from the original sample.
If an intermediate segment contains output format instructions (e.g., ‘You must strictly follow this output format:’), truncate before that line.
Return an empty string if no structure is found."""
    if not isinstance(raw_prompt, str):
        return ""
    tag_idx = raw_prompt.find(TASK_TAG)
    split_idx = raw_prompt.find(SPLIT_TOK)
    if tag_idx == -1 or split_idx == -1 or split_idx <= tag_idx:
        return ""
    middle = raw_prompt[tag_idx + len(TASK_TAG): split_idx]
    sec_idx = middle.find(SECONDARY_SPLIT)
    if sec_idx != -1:
        middle = middle[:sec_idx]
    return middle.strip()


def set_canonical_shell_from_series(series: pd.Series) -> None:
    global CANONICAL_PREFIX, CANONICAL_SUFFIX
    for s in series:
        p, suf, _ = safe_split(s)
        if p is not None:
            CANONICAL_PREFIX = p
            CANONICAL_SUFFIX = suf
            return

# =========================
# 2. LLM Output Analysis & Evaluation Function
# =========================

def parse_label_from_text(text: str) -> str:
    if not isinstance(text, str):
        return "unknown"
    low = text.lower()
    v_match = re.search(r"vaccinated: (\d+)%", low)
    nv_match = re.search(r"not vaccinated: (\d+)%", low)
    if v_match and nv_match:
        v_prob = int(v_match.group(1))
        nv_prob = int(nv_match.group(1))
        if v_prob > nv_prob:
            return "vaccinated"
        elif nv_prob > v_prob:
            return "not vaccinated"
        else:
            return "unknown"
    return "unknown"


def normalize_truth_label(s: str) -> str:
    if not isinstance(s, str):
        return "unknown"
    s = s.lower().strip().replace(".", "").strip()
    if "not vaccinated" in s:
        return "not vaccinated"
    if "vaccinated" in s:
        return "vaccinated"
    return "unknown"


def vaccinated_equality_fn(prediction: tg.Variable, ground_truth_answer: tg.Variable) -> int:
    pred_label = parse_label_from_text(prediction.value)
    true_label = normalize_truth_label(ground_truth_answer.value)
    return int(pred_label == true_label)


def nv_recall_only_fn(prediction: tg.Variable, ground_truth_answer: tg.Variable) -> int:
    pred_label = parse_label_from_text(prediction.value)
    true_label = normalize_truth_label(ground_truth_answer.value)
    if true_label == "not vaccinated":
        return int(pred_label == "not vaccinated")
    return 0


def make_eval_fn(target: str = 'accuracy') -> StringBasedFunction:
    if target == 'nv_recall':
        fn = nv_recall_only_fn
        purpose = "Does the model correctly predict NOT VACCINATED when ground truth is NV?"
    else:
        fn = vaccinated_equality_fn
        purpose = "Does the model correctly predict vaccinated vs not vaccinated?"
    return StringBasedFunction(fn, function_purpose=purpose)

# =========================
# 3. Data loading and partitioning
# =========================

def load_datasets() -> Tuple[List[Tuple[str, str]], List[Tuple[str, str]], List[Tuple[str, str]], pd.DataFrame]:
    df_result = pd.read_excel(RESULT_PATH, usecols=["id", "Task_Prompt", "Vaccination Decision"]).rename(columns={"Vaccination Decision": "prediction"})
    df_truth = pd.read_excel(TRUTH_PATH, usecols=["id", "Task3_Original_Response_Two_Options"]).rename(columns={"Task3_Original_Response_Two_Options": "true_label"})

    df = pd.merge(df_result, df_truth, on="id", how="inner")
    df.dropna(subset=["Task_Prompt", "true_label", "prediction"], inplace=True)

    set_canonical_shell_from_series(df["Task_Prompt"])

    subset_size = min(1000, len(df))
    subset_df = df.sample(n=subset_size, random_state=RANDOM_SEED).reset_index(drop=True)

    if os.path.exists(VAL_SPLIT_SAVE) and os.path.exists(TEST_SPLIT_SAVE):
        val_ids = pd.read_csv(VAL_SPLIT_SAVE)["val_id"].tolist()
        test_ids = pd.read_csv(TEST_SPLIT_SAVE)["test_id"].tolist()
    else:
        train_df, val_and_test_df = train_test_split(subset_df, test_size=0.5, random_state=RANDOM_SEED)
        val_df, test_df = train_test_split(val_and_test_df, test_size=0.5, random_state=RANDOM_SEED)
        val_ids = val_df["id"].tolist()
        test_ids = test_df["id"].tolist()
        pd.DataFrame({"val_id": val_ids}).to_csv(VAL_SPLIT_SAVE, index=False)
        pd.DataFrame({"test_id": test_ids}).to_csv(TEST_SPLIT_SAVE, index=False)

    val_set = [(row["Task_Prompt"], row["true_label"]) for _, row in df[df["id"].isin(val_ids)].iterrows()]
    test_set = [(row["Task_Prompt"], row["true_label"]) for _, row in df[df["id"].isin(test_ids)].iterrows()]
    train_set = [(row["Task_Prompt"], row["true_label"]) for _, row in df[~df["id"].isin(val_ids + test_ids)].iterrows()]

    return train_set, val_set, test_set, df

# =========================
# 4. TextGrad component
# =========================
trainable_prompt = tg.Variable("", requires_grad=True, role_description="trainable reasoning template")


def _clean_trainable_for_merge(tr: str) -> str:
    if not isinstance(tr, str):
        return ""
    filtered = []
    for ln in tr.splitlines():
        low = ln.lower().strip()
        if "% task prompt" in low:
            continue
        if low.startswith("```"):
            continue
        filtered.append(ln)
    txt = "\n".join(filtered)
    m = re.search(r"(?im)^\s*output\s*format\s*:\s*$|" + re.escape(SECONDARY_SPLIT), txt)
    if m:
        txt = txt[:m.start()].rstrip()
    lines = []
    for ln in txt.splitlines():
        if re.search(r"(?i)^\s*likelihood\s+of\s+choosing", ln):
            continue
        lines.append(ln)
    return "\n".join(lines).strip()


def construct_full_prompt(raw_prompt: str) -> str:
    if not isinstance(raw_prompt, str):
        return raw_prompt
    if TASK_TAG not in raw_prompt or SPLIT_TOK not in raw_prompt:
        return raw_prompt
    tag_idx = raw_prompt.index(TASK_TAG)
    split_idx = raw_prompt.index(SPLIT_TOK)
    prefix = raw_prompt[: tag_idx + len(TASK_TAG)].rstrip() + "\n"
    suffix = raw_prompt[split_idx:].lstrip()
    tr_clean = _clean_trainable_for_merge(trainable_prompt.get_value())
    return prefix + tr_clean + ("\n" if tr_clean else "") + suffix


def full_prompt_from_trainable(trainable_text: str) -> str:
    tr_clean = _clean_trainable_for_merge(trainable_text)
    if CANONICAL_PREFIX and CANONICAL_SUFFIX:
        return CANONICAL_PREFIX + tr_clean + ("\n" if tr_clean else "") + CANONICAL_SUFFIX
    return tr_clean


def construct_instruction_prompt(trainable: tg.Variable, initial_prompt: str = None) -> str:
    old_trainable = initial_prompt if initial_prompt is not None else trainable.get_value()
    opt_instruction = f"""You are improving the **reasoning template** shown before a model makes a vaccination decision.
Your task is to **rewrite the template only**. Do **not** produce a decision or percentages.

## Hard constraints
- Do not include any percentages/likelihood/probability or ranking/weighting/scoring instructions in the template.
- Avoid persuasive or rhetorical wording and do not add or duplicate an Output format section.
- Keep language **neutral and symmetric**; avoid persuasive framing.
- **Do not** repeat or modify the task's **Output format** (it appears elsewhere).
- **Do not** mention a disease by name unless it already appears in the provided inputs; otherwise keep it generic (“vaccines”).
- **No personas or demographics**; do not refer to “you”.
- **Preserve the existing section flow** unless simplifying for clarity; keep it concise and bullet-based.
- Use **qualitative markers only** (Low/Medium/High).

## What to optimize
- Begin with disconfirming evidence; keep pro vs. con symmetric; no quotes/citations.
- Add **source-quality checks** and note logical gaps.
- Provide **calibration guidance** when evidence is sparse or contradictory.
- Summarize how the **next step** should weigh signals, **without** giving percentages or decisions.

Rewrite the template below. Return only the new template between tags.

<original>
{old_trainable}
</original>

<new_variable>
[Place the improved reasoning template here; no decisions, no numbers, no output format]
</new_variable>
"""
    return opt_instruction

# =========================
# 5. Evaluation of encapsulation
# =========================

def eval_sample(item: Tuple[str, str], eval_fn: StringBasedFunction, model: tg.BlackboxLLM) -> int:
    raw_prompt, y = item
    full_prompt_text = construct_full_prompt(raw_prompt)
    x = tg.Variable(full_prompt_text, requires_grad=False, role_description="full prompt for model")
    yv = tg.Variable(y, requires_grad=False, role_description="correct answer for the query")
    response = model(x)
    return int(eval_fn({"prediction": response, "ground_truth_answer": yv}).value)


def eval_dataset(dataset: List[Tuple[str, str]], eval_fn: StringBasedFunction, model: tg.BlackboxLLM, max_workers: int = 2) -> List[int]:
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(eval_sample, item, eval_fn, model) for item in dataset]
        return [f.result() for f in concurrent.futures.as_completed(futures)]

# =========================
# 6. Rollback Mechanism
# =========================

def run_validation_revert(system_prompt: tg.Variable,
                          results: dict,
                          current_val_acc: float,
                          offline_baseline: float) -> tuple[bool, float]:
    prev_val = results["validation_acc"][-1]
    if current_val_acc < max(prev_val, offline_baseline):
        system_prompt.set_value(results["prompt"][-1])
        effective_val = prev_val
        accepted = False
        results["validation_acc"].append(prev_val)
    else:
        effective_val = current_val_acc
        accepted = True
        results["validation_acc"].append(current_val_acc)
    results["prompt"].append(system_prompt.get_value())
    return accepted, effective_val

# =========================
# 7. Main Process
# =========================

def main():
    train_set, val_set, test_set, df = load_datasets()
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

    if not trainable_prompt.get_value():
        seed = ""
        for sample, _ in train_set:
            seed = extract_middle_trainable(sample)
            if seed:
                break
        trainable_prompt.set_value(seed)

    llm_api_eval = tg.get_engine("gpt-4o")
    llm_api_test = tg.get_engine("gpt-4o-mini")
    tg.set_backward_engine(llm_api_eval, override=True)

    system_prompt = trainable_prompt
    model = tg.BlackboxLLM(llm_api_test, system_prompt)

    eval_fn = make_eval_fn(EVAL_TARGET)

    df_val_ids = pd.read_csv(VAL_SPLIT_SAVE)["val_id"].tolist()
    df_test_ids = pd.read_csv(TEST_SPLIT_SAVE)["test_id"].tolist()

    df_val = df[df["id"].isin(df_val_ids)]
    df_test = df[df["id"].isin(df_test_ids)]

    val_pred = df_val["prediction"].str.lower().str.strip()
    val_true = df_val["true_label"].str.lower().str.strip()
    initial_val_acc_offline = accuracy_score(val_true, val_pred)

    test_pred = df_test["prediction"].str.lower().str.strip()
    test_true = df_test["true_label"].str.lower().str.strip()
    initial_test_acc_offline = accuracy_score(test_true, test_pred)

    print(f"[OFFLINE] Initial val acc: {initial_val_acc_offline:.4f}, test acc: {initial_test_acc_offline:.4f}")

    val_acc = float(np.mean(eval_dataset(val_set, eval_fn, model)))
    test_acc = float(np.mean(eval_dataset(test_set, eval_fn, model)))
    print(f"[ONLINE] Initial val acc: {val_acc:.4f}, test acc: {test_acc:.4f}")

    results = {
        "test_acc": [test_acc],
        "validation_acc": [val_acc],
        "prompt": [trainable_prompt.get_value()],
        "offline_val_acc": [initial_val_acc_offline],
        "offline_test_acc": [initial_test_acc_offline],
    }

    print(f"Initial val acc: {val_acc:.4f}, test acc: {test_acc:.4f}")

    full_prompt_example = construct_full_prompt(train_set[0][0])
    trainable_initial = trainable_prompt.get_value()
    print("===== Initial Full Prompt Example =====")
    print(full_prompt_example)
    print("\n===== Initial Trainable Prompt (seeded from source) =====")
    print(trainable_initial)

    optimizer = tg.TextualGradientDescent(engine=llm_api_eval, parameters=[trainable_prompt])

    best_val_acc = 0.0
    best_trainable_prompt = trainable_prompt.get_value()
    best_test_acc = test_acc

    for epoch in range(EPOCHS):
        pbar = tqdm(train_loader, position=0)
        for steps, (batch_x, batch_y) in enumerate(pbar):
            pbar.set_description(f"\nTraining step {steps}. Epoch {epoch}")

            opt_instruction = construct_instruction_prompt(trainable_prompt)
            optimizer.optimizer_prompt = tg.Variable(opt_instruction, requires_grad=False, role_description="instruction prompt")

            optimizer.zero_grad()
            losses = []
            for x_text, y in zip(batch_x, batch_y):
                full_prompt = construct_full_prompt(x_text)
                x = tg.Variable(full_prompt, requires_grad=False, role_description="full prompt for model")
                yv = tg.Variable(y, requires_grad=False, role_description="correct answer for the query")
                response = model(x)
                loss = eval_fn({"prediction": response, "ground_truth_answer": yv})
                losses.append(loss)

            total_loss = tg.sum(losses)
            total_loss.backward()

            optimizer.step()

            try:
                _cur = trainable_prompt.get_value()
                _cur_clean = _clean_trainable_for_merge(_cur)
                if _cur_clean != _cur:
                    trainable_prompt.set_value(_cur_clean)
            except Exception:
                pass

            val_scores = eval_dataset(val_set, eval_fn, model)
            test_scores = eval_dataset(test_set, eval_fn, model)
            val_acc = float(np.mean(val_scores))
            test_acc = float(np.mean(test_scores))

            accepted, eff_val = run_validation_revert(trainable_prompt, results, val_acc, results["offline_val_acc"][0])
            results["test_acc"].append(test_acc)

            status = "ACCEPTED" if accepted else "REJECTED"
            print(f"\n[STEP] {status} | val_acc(cur={val_acc:.4f}, eff={eff_val:.4f}) | test_acc={test_acc:.4f}")

            print("\nUpdated trainable template:\n", trainable_prompt.get_value())
            try:
                preview_full = construct_full_prompt(batch_x[0])
                # print("\nFull prompt preview (keeps % Task Prompt & Output format):\n", preview_full[:1200])
            except Exception:
                pass

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_trainable_prompt = trainable_prompt.get_value()
                best_test_acc = test_acc
                best_full_prompt = full_prompt_from_trainable(best_trainable_prompt)
                print("New best prompt found!")

                try:
                    with open(BEST_PROMPT_PATH, "w") as f:
                        f.write(best_trainable_prompt)
                    with open(BEST_FULL_PROMPT_PATH, "w") as f:
                        f.write(best_full_prompt)
                    with open(BEST_ACCURACY_PATH, "w") as f:
                        f.write(f"Best Val Acc: {best_val_acc:.4f}\n")
                        f.write(f"Corresponding Test Acc: {best_test_acc:.4f}\n")
                except Exception as e:
                    print("Warning: failed to save best prompt/accuracy:", e)

            if steps >= (MAX_STEPS_PER_EPOCH - 1):
                break

    try:
        with open(BEST_PROMPT_PATH, "w") as f:
            f.write(best_trainable_prompt)
        with open(BEST_FULL_PROMPT_PATH, "w") as f:
            f.write(full_prompt_from_trainable(best_trainable_prompt))
        with open(BEST_ACCURACY_PATH, "w") as f:
            f.write(f"Best Val Acc: {best_val_acc:.4f}\n")
            f.write(f"Corresponding Test Acc: {best_test_acc:.4f}\n")
    except Exception as e:
        print("Warning: failed to save final best prompt/accuracy:", e)

    print("\nTraining finished.")
    print(f"Best Val Acc: {best_val_acc:.4f}; Corresponding Test Acc: {best_test_acc:.4f}")


if __name__ == "__main__":
    main()


[OFFLINE] Initial val acc: 0.8080, test acc: 0.8080
[ONLINE] Initial val acc: 0.8560, test acc: 0.8400
Initial val acc: 0.8560, test acc: 0.8400
===== Initial Full Prompt Example =====
[Task Information]: You are a 27-year-old male. You do not hold a university degree and your income is above the country median.
% Media Diet Information
Assume today is June 29, 2022. Your decision about receiving a COVID-19 vaccine is primarily shaped by your demographics and prior beliefs. However, the kind of information you are exposed to in the media may also influence your thinking.
Below are five real-world news articles that reflect the type of media you are likely to encounter.
As you read them, consider:
- Do any of these articles reinforce your existing beliefs?
- Do any contradict them or introduce new concerns or reassurances?
Unless strongly challenged by the articles, your decision should remain aligned with your prior beliefs. Only adjust your position if a particular article clearly shi

0it [00:00, ?it/s]
Training step 0. Epoch 0: : 0it [00:00, ?it/s]


[STEP] ACCEPTED | val_acc(cur=0.8880, eff=0.8880) | test_acc=0.8640

Updated trainable template:
 Now, based on your demographics and the content of the five articles above, answer whether you would like to receive the COVID-19 vaccine. Select one from [<Vaccinated>: Believe vaccination is important for protection against COVID-19; <Not Vaccinated>: Decide not to take the vaccine due to concerns about necessity, effectiveness, or risks.] 

To guide your decision, follow these steps:

1. **Evidence Extraction**: Identify and mention at least two specific data points, statistics, or expert opinions from the articles that support your decision. 

2. **Counterargument Analysis**: List any factors or arguments presented in the articles that might challenge your decision and explain why they do or do not change your stance.

3. **Contextual Relevance**: Highlight any recent developments, public health policies, or societal attitudes mentioned in the articles that are relevant to your decisi


Training step 0. Epoch 0: : 1it [08:15, 495.12s/it]
Training step 1. Epoch 0: : 1it [08:15, 495.12s/it]
Training step 1. Epoch 0: : 2it [17:56, 546.04s/it]
Training step 2. Epoch 0: : 2it [17:56, 546.04s/it]


[STEP] ACCEPTED | val_acc(cur=0.8960, eff=0.8960) | test_acc=0.8600

Updated trainable template:
 Now, based on your demographics and the content of the five articles above, answer whether you would like to receive the COVID-19 vaccine. Select one from [<Vaccinated>: Vaccination is essential for protection against COVID-19; <Not Vaccinated>: Decide not to take the vaccine due to concerns about necessity, effectiveness, or risks.]

To guide your decision, follow these steps:

1. **Evidence Extraction**: Identify and mention at least two specific data points, statistics, or expert opinions from the articles that support your decision. Include specific health benefits or protective effects of vaccination, such as protection against severe illness or reduction in transmission rates.

2. **Counterargument Analysis**: List any factors or arguments presented in the articles that might challenge your decision and provide evidence-based responses to these points, addressing common misconceptio

...


[STEP] REJECTED | val_acc(cur=0.9040, eff=0.9160) | test_acc=0.8400

Updated trainable template:
 Now, based on your demographics and the content of the five articles above, answer whether you would like to receive the COVID-19 vaccine. Select one from [<Vaccinated>: Vaccination is essential for protection against COVID-19; <Not Vaccinated>: Decide not to take the vaccine due to concerns about necessity, effectiveness, or risks.]

To guide your decision, follow these steps:

1. **Balanced Evidence Extraction**: Identify and mention at least two specific data points, statistics, or expert opinions from the articles that support your decision. Include direct quotes or paraphrased key points that clearly demonstrate both the health benefits and potential risks of vaccination, such as protection against severe illness or concerns about side effects.

2. **Explicit Counterargument Section**: List at least two factors or arguments presented in the articles that might challenge your decision


Training step 1. Epoch 9: : 2it [21:30, 639.37s/it]
Training step 2. Epoch 9: : 2it [21:30, 639.37s/it]


[STEP] REJECTED | val_acc(cur=0.9000, eff=0.9160) | test_acc=0.8280

Updated trainable template:
 Now, based on your demographics and the content of the five articles above, answer whether you would like to receive the COVID-19 vaccine. Select one from [<Vaccinated>: Vaccination is essential for protection against COVID-19; <Not Vaccinated>: Decide not to take the vaccine due to concerns about necessity, effectiveness, or risks.]

To guide your decision, follow these steps:

1. **Balanced Evidence Extraction**: Identify and mention at least two specific data points, statistics, or expert opinions from the articles that support your decision. Include direct quotes or paraphrased key points that clearly demonstrate both the health benefits and potential risks of vaccination, such as protection against severe illness or concerns about side effects.

2. **Explicit Counterargument Section**: List at least two factors or arguments presented in the articles that might challenge your decision


Training step 2. Epoch 9: : 2it [31:35, 947.63s/it]


[STEP] REJECTED | val_acc(cur=0.8960, eff=0.9160) | test_acc=0.8400

Updated trainable template:
 Now, based on your demographics and the content of the five articles above, answer whether you would like to receive the COVID-19 vaccine. Select one from [<Vaccinated>: Vaccination is essential for protection against COVID-19; <Not Vaccinated>: Decide not to take the vaccine due to concerns about necessity, effectiveness, or risks.]

To guide your decision, follow these steps:

1. **Balanced Evidence Extraction**: Identify and mention at least two specific data points, statistics, or expert opinions from the articles that support your decision. Include direct quotes or paraphrased key points that clearly demonstrate both the health benefits and potential risks of vaccination, such as protection against severe illness or concerns about side effects.

2. **Explicit Counterargument Section**: List at least two factors or arguments presented in the articles that might challenge your decision

In [ ]:
# Read the optimal prompt
with open("/content/drive/MyDrive/LLM Vaccination Attitudes/Results/RAG/best_prompt_250.txt", "r") as f:
    best_prompt_text = f.read()

# Replace current trainable_prompt
trainable_prompt.set_value(best_prompt_text)


In [ ]:
trainable_prompt.get_value()

'```\nStructured Reasoning Framework:\n1. Key Points from Articles:\n   - Identify and summarize key data points or expert opinions from the articles that support your decision.\n   - Provide direct quotes or detailed summaries of the most relevant parts of each article.\n   - List any statistical data or expert opinions that support vaccination.\n\n2. Alignment with Existing Beliefs:\n   - Explicitly state your existing beliefs regarding vaccination at the beginning of the reasoning.\n   - Explicitly state your current vaccination status and any reasons for this status.\n   - Assess how the articles align or contradict these beliefs.\n\n3. Quantitative Analysis:\n   - Summarize any quantitative data from the articles that supports your decision and explain its impact on your likelihood percentages.\n   - Introduce a formulaic approach to link each article\'s evidence to the likelihood percentages.\n   - Rate the strength of the evidence in each article on a scale from 1 to 5, and adju

# TextGrad on M2

In [ ]:
import pandas as pd

BEST_PROMPT_PATH = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/RAG/best_prompt_250_v3.txt"
ORIGINAL_XLSX = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/task_inputs_two_options_four_diets_M2_4omini_seed1000.xlsx"
OUTPUT_XLSX = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/TextGrad_v3_task_inputs_two_options_four_diets_M2_4omini_seed1000.xlsx"

TASK_TAG = "% Task Prompt"
SPLIT_TOK = "---\nOutput format:"

with open(BEST_PROMPT_PATH, "r") as f:
    best_prompt_text = f.read().strip()

df = pd.read_excel(ORIGINAL_XLSX)

def replace_task_prompt(full_prompt, new_prompt):
    if not isinstance(full_prompt, str):
        return full_prompt
    if TASK_TAG not in full_prompt or SPLIT_TOK not in full_prompt:
        return full_prompt
    tag_idx = full_prompt.index(TASK_TAG) + len(TASK_TAG)
    split_idx = full_prompt.index(SPLIT_TOK)
    prefix = full_prompt[:tag_idx].rstrip() + "\n"
    suffix = full_prompt[split_idx:].lstrip()
    return prefix + new_prompt.strip() + "\n" + suffix

df["Task_Prompt_Full"] = df["Task_Prompt_Full"].apply(lambda x: replace_task_prompt(x, best_prompt_text))

df.to_excel(OUTPUT_XLSX, index=False)

print(f"Finished. File saved to: {OUTPUT_XLSX}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

filtered_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/test_500/Chatgpt/Three_Task_Original_Response.xlsx"
results_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/M2/Media_four_diets/Task_Results_Two_Options_Four_Diets_M2_4omini_seed1000.xlsx"

filtered_df = pd.read_excel(filtered_path)
task_results_df = pd.read_excel(results_path)

filtered_df["id"] = filtered_df["id"].astype(int)
task_results_df["id"] = task_results_df["id"].astype(int)

merged_df = task_results_df.merge(
    filtered_df[["id", "Task3_Original_Response_Two_Options"]],
    on="id",
    how="inner"
)

merged_df["true_label"] = merged_df["Task3_Original_Response_Two_Options"]
merged_df["predicted_label"] = merged_df["Vaccination Decision"]

print("Label Set:", sorted(set(merged_df["true_label"]) | set(merged_df["predicted_label"])))

eps = 1e-10
merged_df["prob_vaccinated"] = merged_df["Likelihood of Choosing Vaccinated (%)"].str.replace("%", "").astype(float) / 100
merged_df["true_binary"] = merged_df["true_label"].apply(lambda x: 1 if x == "Vaccinated" else 0)

merged_df["cross_entropy"] = -(
    merged_df["true_binary"] * np.log(merged_df["prob_vaccinated"] + eps) +
    (1 - merged_df["true_binary"]) * np.log(1 - merged_df["prob_vaccinated"] + eps)
)

entropy_by_class = merged_df.groupby("true_label")["cross_entropy"].mean()

report = classification_report(
    merged_df["true_label"],
    merged_df["predicted_label"],
    labels=["Vaccinated", "Not Vaccinated"],
    target_names=["Vaccinated", "Not Vaccinated"],
    digits=4
)
print("\nBefore TextGrad:")
# print("\nClassification Report:")
print(report)

print("\nBinary Cross Entropy Loss by Class:")
for label, loss in entropy_by_class.items():
    print(f"{label}: {loss:.4f}")

In [ ]:
# After Textgrad
# 3:1
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

filtered_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/test_500/Chatgpt/Three_Task_Original_Response.xlsx"
results_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/M2/Media_four_diets/3_1_TextGrad_v3_Task_Results_Two_Options_Four_Diets_M2_4omini_seed1000.xlsx"

filtered_df = pd.read_excel(filtered_path)
task_results_df = pd.read_excel(results_path)

filtered_df["id"] = filtered_df["id"].astype(int)
task_results_df["id"] = task_results_df["id"].astype(int)

merged_df = task_results_df.merge(
    filtered_df[["id", "Task3_Original_Response_Two_Options"]],
    on="id",
    how="inner"
)

merged_df["true_label"] = merged_df["Task3_Original_Response_Two_Options"]
merged_df["predicted_label"] = merged_df["Vaccination Decision"]

print("Label Set:", sorted(set(merged_df["true_label"]) | set(merged_df["predicted_label"])))

eps = 1e-10
merged_df["prob_vaccinated"] = merged_df["Likelihood of Choosing Vaccinated (%)"].str.replace("%", "").astype(float) / 100
merged_df["true_binary"] = merged_df["true_label"].apply(lambda x: 1 if x == "Vaccinated" else 0)

merged_df["cross_entropy"] = -(
    merged_df["true_binary"] * np.log(merged_df["prob_vaccinated"] + eps) +
    (1 - merged_df["true_binary"]) * np.log(1 - merged_df["prob_vaccinated"] + eps)
)

entropy_by_class = merged_df.groupby("true_label")["cross_entropy"].mean()

report = classification_report(
    merged_df["true_label"],
    merged_df["predicted_label"],
    labels=["Vaccinated", "Not Vaccinated"],
    target_names=["Vaccinated", "Not Vaccinated"],
    digits=4
)
print("\nw_nv:w_v = 3:1")
print("\nAfter TextGrad:")
# print("\nClassification Report:")
print(report)

print("\nBinary Cross Entropy Loss by Class:")
for label, loss in entropy_by_class.items():
    print(f"{label}: {loss:.4f}")